# K-Nearest Neigbor Based Recommender System

In [1]:
# (https://grouplens.org/datasets/movielens/) ml-100k.zip (size: 5 MB)

In [1]:
import pandas as pd

In [2]:
ratings=pd.read_csv("ml-100k/u.data", sep="\t",header=None,names=["user_id","movie_id","rating"],usecols=range(3)) #csv de ki boşluk ya da tab kullanılan alanları seperatörleri düzenler

In [3]:
ratings.head()

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [4]:
range(3)

range(0, 3)

In [7]:
movies=pd.read_csv("ml-100k/u.item", sep="|",header=None,encoding="iso-8859-1") 

In [8]:
movies.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [10]:
import numpy as np

movieProperties = ratings.groupby("movie_id").agg({"rating": [np.size, np.mean]})
movieProperties.head()
#en çok yorum alan filmlerin ortalamaları

rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

In [12]:
movieNumRatings = pd.DataFrame(movieProperties["rating"]["size"])
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x-np.min(x)) / (np.max(x) - np.min(x)))
movieNormalizedNumRatings.head()
#yüzdelerinin hesaplanması

,size
movie_id,
1,0.774914
2,0.223368
3,0.152921
4,0.357388
5,0.146048


In [13]:
movieDict = {}
with open(r'ml-100k/u.item', encoding="iso-8859-1") as f:
    
    for line in f:
        fields = line.rstrip('\n').split('|')
        movieID = int(fields[0])
        name = fields[1]
        genres = fields[5:24] #kategori 5 ten 24 e kadar olan sütunlar
        genres = list(map(int, genres))
        movieDict[movieID] = (name, genres, movieNormalizedNumRatings.loc[movieID].get('size'), movieProperties.loc[movieID].rating.get('mean'))


In [15]:
from scipy import spatial

def ComputeDistance(a, b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.cosine(genresA, genresB) #cos uzaklığı 
    popularityA = a[2]
    popularityB = b[2]
    popularitiyDistance = abs(popularityA - popularityB) #bir pop un diğer pop a uzaklığı
    return genreDistance + popularitiyDistance


In [16]:
import operator

def getNeighbors(movieID, K):
    distances = []
    for movie in movieDict:
        if (movie != movieID):
            dist = ComputeDistance(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

K = 30
avgRating = 0
neighbors = getNeighbors(1, K)
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print( movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]))
#aralarında ki uzaklığın en küçük olanlar listelenir. Birbirine en yakın filmler listesi

Liar Liar (1997) 3.156701030927835
Aladdin (1992) 3.8127853881278537
Willy Wonka and the Chocolate Factory (1971) 3.6319018404907975
Monty Python and the Holy Grail (1974) 4.0664556962025316
Full Monty, The (1997) 3.926984126984127
George of the Jungle (1997) 2.685185185185185
Beavis and Butt-head Do America (1996) 2.7884615384615383
Birdcage, The (1996) 3.4436860068259385
Home Alone (1990) 3.0875912408759123
Lion King, The (1994) 3.7818181818181817
Aladdin and the King of Thieves (1996) 2.8461538461538463
Jungle2Jungle (1997) 2.4393939393939394
Babe (1995) 3.9954337899543377
Wrong Trousers, The (1993) 4.466101694915254
Raising Arizona (1987) 3.875
Beauty and the Beast (1991) 3.792079207920792
Back to the Future (1985) 3.834285714285714
101 Dalmatians (1996) 2.908256880733945
Fish Called Wanda, A (1988) 3.785425101214575
Pinocchio (1940) 3.6732673267326734
Matilda (1996) 3.210526315789474
In & Out (1997) 3.3043478260869565
Mary Poppins (1964) 3.7247191011235956
Fantasia (1940) 3.770114